# Running on GPUs

There are differnt options to run your code on GPUs. The most common ones are:

- **Google Colab**: A free cloud service that provides Jupyter notebooks with GPU support. You can use it to run your code on a GPU without any setup.
- **Local GPU**: If you have a GPU on your local machine, you can run your code on it using PyTorch or TensorFlow. You need to install the appropriate drivers and libraries to use the GPU.
- **Cloud GPU**: You can rent a GPU from a cloud provider like AWS, Google Cloud, or Azure. This option is more expensive but allows you to use powerful GPUs for your projects.

In [16]:
import torch

In [17]:
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

PyTorch version: 2.6.0+cu126
CUDA available: True
CUDA version: 12.6


To make the code device agnostic, the code below (from [PyTorch documentation](https://pytorch.org/docs/stable/notes/cuda.html#cuda-semantics)) checks if CUDA is available and sets the device accordingly. If CUDA is not available, it defaults to CPU. This way, you can run the same code on both CPU and GPU without any changes.

In [ ]:
# Do not run this code here

import argparse
# import torch

parser = argparse.ArgumentParser(description='PyTorch Example')
parser.add_argument('--disable-cuda', action='store_true',
                    help='Disable CUDA')
args = parser.parse_args()
args.device = None
if not args.disable_cuda and torch.cuda.is_available():
    args.device = torch.device('cuda')
else:
    args.device = torch.device('cpu')

# Example usage (not in the original code)
#x = torch.empty((8, 42), device=args.device)
# net = Network().to(device=args.device)

A simpler implementation for smaller projects:

In [23]:
# Set device type
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

Let's create a tensor and check its device:

In [24]:
tensor = torch.tensor([1, 2, 3])
print(tensor, tensor.device)

tensor([1, 2, 3]) cpu


Let's now move the tensor to the GPU:

In [25]:
tensor = tensor.to(device)
print(tensor, tensor.device)

tensor([1, 2, 3], device='cuda:0') cuda:0


For some computations, for instance when using Numpy, we might need to move the tensor back to CPU. The `.cpu()` will copy the tensor to the CPU memory.

In [26]:
tensor_to_cpu = tensor.cpu().numpy()
print(tensor_to_cpu, tensor_to_cpu.device)

[1 2 3] cpu


The original tensor is still on the GPU, and we can check its device:

In [27]:
print(tensor.device)

cuda:0


Let's now do some tensor operations.

In [28]:
A = torch.rand(7, 7)
A

tensor([[0.9989, 0.2149, 0.0426, 0.9434, 0.8821, 0.6689, 0.2994],
        [0.7143, 0.1341, 0.0160, 0.4524, 0.2977, 0.6134, 0.6843],
        [0.4712, 0.2895, 0.7856, 0.2600, 0.8902, 0.4731, 0.6097],
        [0.7839, 0.3622, 0.5973, 0.9278, 0.0310, 0.1695, 0.0028],
        [0.4553, 0.1025, 0.3298, 0.8413, 0.5296, 0.4962, 0.9753],
        [0.4377, 0.5322, 0.4453, 0.0715, 0.5955, 0.7760, 0.5759],
        [0.4143, 0.2822, 0.3901, 0.3967, 0.6536, 0.2884, 0.9868]])

In [30]:
B = torch.rand(1, 7)
B

tensor([[0.2236, 0.8643, 0.2792, 0.3037, 0.4342, 0.8267, 0.0259]])

In [32]:
C = torch.matmul(A, B.T)
C

tensor([[1.6511],
        [1.0715],
        [1.4472],
        [1.0905],
        [1.2033],
        [1.6189],
        [1.1136]])

In [33]:
RANDOM_SEED = 0
torch.manual_seed(RANDOM_SEED)
A = torch.rand(7, 7)
torch.manual_seed(RANDOM_SEED)
B = torch.rand(1, 7)

C = torch.matmul(A, B.T)
C

tensor([[1.5985],
        [1.1173],
        [1.2741],
        [1.6838],
        [0.8279],
        [1.0347],
        [1.2498]])

In [53]:
CUDA_SEED = 1234
torch.cuda.manual_seed(CUDA_SEED)
D = torch.rand(2, 3).to(device)
torch.cuda.manual_seed(CUDA_SEED)
E = torch.rand(2, 3).to(device)
F = torch.matmul(D, E.T).cpu()
F

tensor([[0.4410, 0.3345],
        [0.7856, 0.5843]])

In [54]:
torch.min(F), torch.max(F)

(tensor(0.3345), tensor(0.7856))

In [55]:
torch.argmin(F), torch.argmax(F)

(tensor(1), tensor(2))

In [62]:
torch.manual_seed(7)
G = torch.rand(1, 1, 1, 10)
print(G)
print(G.shape)

tensor([[[[0.5349, 0.1988, 0.6592, 0.6569, 0.2328, 0.4251, 0.2071, 0.6297,
           0.3653, 0.8513]]]])
torch.Size([1, 1, 1, 10])


In [63]:
G_squeeze = G.squeeze()
print(G_squeeze)
print(G_squeeze.shape)

tensor([0.5349, 0.1988, 0.6592, 0.6569, 0.2328, 0.4251, 0.2071, 0.6297, 0.3653,
        0.8513])
torch.Size([10])
